In [4]:
!pip install torch
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.8 MB/s eta 0:00:0000:0100:01


In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker
...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,Male,Never smoked
1545,ID00426637202313170790466,19,2978,73.126412,73,Male,Never smoked
1546,ID00426637202313170790466,31,2908,71.407524,73,Male,Never smoked
1547,ID00426637202313170790466,43,2975,73.052745,73,Male,Never smoked


In [3]:
y = train["FVC"]
train_df = train.copy()
train_df=train_df.drop(columns = ["FVC","Patient"])
train_df['Sex'] = train_df['Sex'].map(lambda x: 1 if(x == "Male") else 0 )
train_df['SmokingStatus'] = train_df['SmokingStatus'].map(lambda x: 1 if(x == "Ex-smoker") else 0 )
train_df = train_df.astype('double')
train_df

,Weeks,Percent,Age,Sex,SmokingStatus
0,-4.0,58.253649,79.0,1.0,1.0
1,5.0,55.712129,79.0,1.0,1.0
2,7.0,51.862104,79.0,1.0,1.0
3,9.0,53.950679,79.0,1.0,1.0
4,11.0,52.063412,79.0,1.0,1.0
...,...,...,...,...,...
1544,13.0,66.594637,73.0,1.0,0.0
1545,19.0,73.126412,73.0,1.0,0.0
1546,31.0,71.407524,73.0,1.0,0.0
1547,43.0,73.052745,73.0,1.0,0.0


In [4]:
batch_size = 100
n_iters = 3000
num_epochs = 150


In [2]:
class FNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        #self.input_dim, self.hidden_dim, self.output_dim = 5, 50, 1

        super(FNN, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
        
#         self.model = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim),
#               nn.ReLU(),
#               nn.Linear(self.hidden_dim, self.output_dim),
#               nn.Sigmoid())
        
#         self.loss_function = nn.MSELoss()
#         self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.learning_rate)

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)
        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

NameError: name 'nn' is not defined

In [3]:
input_dim, hidden_dim, output_dim = 5, 50, 1
model = FNN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()


NameError: name 'FNN' is not defined

In [63]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [64]:
num_epochs = 2

In [74]:
#data_x = torch.tensor([row['Weeks'], row['Percent'],row['Age'],row['Sex'],row['SmokingStatus']])
data_x = torch.from_numpy(train_df.to_numpy())
#torch.flatten(data_x)
data_x = data_x.to(torch.float32)
data_y = torch.tensor(y.values)
data_y = data_y.to(torch.float32)
data_x.shape

torch.Size([1549, 5])

In [73]:
iter = 0

for epoch in range(num_epochs):
    model.train()
    for index, row in train_dl.iterrows():
        preds = model(data_x)
        loss_func = nn.MSELoss()
        loss = loss_func(preds, data_y)

        opt = torch.optim.SGD(model.parameters(), lr=learning_rate) 
        loss.backward()
        opt.step()
        opt.zero_grad()

    print(model.eval())

/Users/maxwang/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1549])) that is different to the input size (torch.Size([1549, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


FeedforwardNeuralNetModel(
  (fc1): Linear(in_features=5, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)
FeedforwardNeuralNetModel(
  (fc1): Linear(in_features=5, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)


In [67]:
test_x = test.drop(columns = ['FVC', "Patient"])
test_x['Sex'] = test_x['Sex'].map(lambda x: 1 if(x == "Male") else 0 )
test_x['SmokingStatus'] = test_x['SmokingStatus'].map(lambda x: 1 if(x == "Ex-smoker") else 0 )
test_y = test["FVC"]

test_features = torch.from_numpy(test_x.to_numpy())
test_features = test_features.to(torch.float32)
ytest = torch.tensor(test_y.values)
ytest = ytest.to(torch.float32)


In [68]:
model(test_features)

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)